In [1]:
# --- necessary libraries ---
import pickle
import numpy as np
import pandas as pd

from pprint import pprint

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from tqdm import tqdm, tqdm_notebook
from time import sleep
import time

## 001. Prepared data import / Импорт предобработанных данных

In [3]:
path_df = "[02].Data_tfidf/df_final.pickle"
with open(path_df, 'rb') as data:
    df_news = pickle.load(data)

- 20% of original data / оригинальных данных
- data is lovercased / данные приведены к нижнему регистру
- Russian stop-words were dropped / изъяты русскоязычные стоп-слова
- special characters and punctuation signs were dropped / изъяты служебные символы и знаки пунктации
- lemmatization is used / произведена лемматизация

In [185]:
len(df_news)


--- 20% оригинальных данных
--- данные приведены к нижнему регистру
--- изъяты русскоязычные стоп-слова
--- изъяты служебные символы и знаки пунктации
--- произведена лемматизация



147793

In [186]:
print('''
--- количество уникальных категорий
''')
len(df_news['topic'].unique())


--- количество уникальных категорий



20

In [187]:
df_news.head(2)

,title,topic,text_final,year,topic_id
0,Вместо олимпийской деревни в Лужниках построят...,Дом,ожидание москва выиграть конкурс право принима...,2006,10
1,В районе Московского НПЗ произошел взрыв,Россия,район московский нефтеперерабатывающий завод а...,2015,0


In [8]:
# --- text sample ---
df_news.text_final[1][:1000]

'район московский нефтеперерабатывающий завод акватория москвырек произойти взрыв последующий возгорание нефтепродукт сообщить лентер компания транснефть район дно река проходить кольцевой нефтепродуктопровод который состоять три ветка авиационный керосин дизельный топливо бензин данный момент трубопровод остановить компания заявить устанавливать причина произойти место работать ремонтный бригада мчс россия лентер сообщить угроза экология город здоровье население спасатель выехать место происшествие интерфакс передавать москверек загореться пятно нефтепродукт источник экстренный служба сообщить агентство вечером август район улица поречной произойти возгорание трава пламя перекинуться поверхность вода информация прессслужба гу мчс москва площадь пожар составлять квадратный метр'

### 1.1 Removing outlying categories with poor amount of data <br>Удаление категорий-выбросов, где данных для обучения катастрофически мало.

In [188]:
df_news['topic'].unique()

array(['Дом', 'Россия', 'Экономика', 'Спорт', 'Мир', 'Бывший СССР',
       'Наука и техника', 'Культура', 'Силовые структуры', 'Из жизни',
       'Путешествия', 'Интернет и СМИ', 'Бизнес', 'Ценности',
       '69-я параллель', 'Культпросвет ', 'Крым', 'Библиотека', 'Оружие',
       'Легпром'], dtype=object)

In [8]:
%%time

df_news = df_news[(df_news['topic_id'] != 12) 
                  & (df_news['topic_id'] != 14) 
                  & (df_news['topic_id'] != 16) 
                  & (df_news['topic_id'] != 17)
                  & (df_news['topic_id'] != 18)
                  & (df_news['topic_id'] != 21)]
df_news['topic_id'].unique()

Wall time: 57.8 ms


array([ 0,  2,  9, 20, 10,  4,  5,  1,  6,  3, 13,  7,  8, 15],
      dtype=int64)

In [9]:
df_news['topic'].unique()

array(['Дом', 'Россия', 'Экономика', 'Спорт', 'Мир', 'Бывший СССР',
       'Наука и техника', 'Культура', 'Силовые структуры', 'Из жизни',
       'Путешествия', 'Интернет и СМИ', 'Бизнес', 'Ценности',
       '69-я параллель', 'Культпросвет ', 'Крым', 'Библиотека', 'Оружие',
       'Легпром'], dtype=object)

## 2. doc2vec model trainig / Обучение модели doc2vec

In [10]:
# --- main libraries ---
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE

In [11]:
from nltk.tokenize import word_tokenize

### 2.1. Train & Test split / Разбиение на обучающую и тестовую выборки
doc2vec takes a special TaggedDocumen object as input for training<br>
doc2vec для обучения модели принимает тэгированный объект TaggedDocument 

In [13]:
%%time
# --- train and test split ---
train, test = train_test_split(df_news.reset_index(drop=True), test_size=.15, random_state=5)

tagged_tr = [TaggedDocument(words=word_tokenize(_d.lower()),\
tags=[str(i)]) for i, _d in tqdm_notebook(enumerate(train.text_final))]

# --- tagging ---
tagged_test = [TaggedDocument(words=word_tokenize(_d.lower()),\
tags=[str(i)]) for i, _d in tqdm_notebook(enumerate(test.text_final))]


Wall time: 1min 47s


### 2.2 Hyperparameters set for doc2vec model / Установка гиперпараметров модели doc2vec

- <b>vector_size</b> - vector dimensionality / размерность вектора
- <b>window</b> - sliding window size / размер скользящего окна
- <b>alpha</b> - start working velocity / начальная скорость работы модели
- <b>min_alpha</b> - minimal velocity / минимальная скорость
- <b>min_count</b> - minimal count
- <b>dm</b> - training algorithm / алгоритм обучения модели - Distributed Memory/Распределенная Память
- <b>hs</b> - hierarchical softmax optimization / оптимизация посредством функции иерархического софтмакса
- <b>workers</b> - kernel amount / количество ядер
- <b>tagged_tr</b> - test set internal vocabulary / словарь тестовой выборки

In [15]:
%%time
# --- parameters for doc2vec ---

modeld2v = Doc2Vec(vector_size     = 100,          #100 should be fine based on the standards
                window             = 5, 
                alpha              = .065,         #initial learning rate
                min_alpha          = 0.00025,      #learning rate drops linearly to this
                min_count          = 1,            #ignores all words with total frequency lower than this.
                dm                 = 1,            #algorith 1=distributed memory 0=PV-DBOW
                hs                 = 1,
               # negative           = 10,
                workers            = 4)            #cores to use

# построение словаря слов тестовой выборки
modeld2v.build_vocab(tagged_tr)

Wall time: 15.1 s


### 2.3 Model training / Обучение модели Doc2vec, 60 epochs

- <b>max_epochs</b> - number of model iterations / количество итераций модели
- <b>modeld2v</b> - trained doc2vec model / обученная doc2vec модель

In [16]:
%%time

max_epochs = 60

start = time.time()
t1 = time.time()

for epoch in tqdm_notebook(range(max_epochs)):
    print('Шаг № {0}'.format(epoch+1))
    modeld2v.train(tagged_tr,
                   total_examples=modeld2v.corpus_count, 
                   epochs=modeld2v.epochs)
    
    # --- decrease training velocity per iteration / уменьшение скорости обучения при каждой итерации ---
    modeld2v.alpha -= 0.0002
    
    # --- a little correction / поправка на скорости обучения ---
    modeld2v.min_alpha = modeld2v.alpha
    
    stop_time = time.time()
    print(f"Время обучения: {stop_time - start} сек")

    sleep(0.01)

print("Готово! Модель отработала")

# --- save model ---
t2 = time.time()    
modeld2v.save("147knews.model")

print("Время работы модели Doc2Vec: {}".format(t2-t1))

Шаг № 1
Время обучения: 126.86310958862305 сек
Шаг № 2
Время обучения: 254.22460985183716 сек
Шаг № 3
Время обучения: 385.32250690460205 сек
Шаг № 4
Время обучения: 515.3967525959015 сек
Шаг № 5
Время обучения: 645.4821002483368 сек
Шаг № 6
Время обучения: 778.3719022274017 сек
Шаг № 7
Время обучения: 907.7246084213257 сек
Шаг № 8
Время обучения: 1036.9598081111908 сек
Шаг № 9
Время обучения: 1165.8913662433624 сек
Шаг № 10
Время обучения: 1294.8225629329681 сек
Шаг № 11
Время обучения: 1423.7815823554993 сек
Шаг № 12
Время обучения: 1553.3730566501617 сек
Шаг № 13
Время обучения: 1683.4823551177979 сек
Шаг № 14
Время обучения: 1812.2397124767303 сек
Шаг № 15
Время обучения: 1941.1073777675629 сек
Шаг № 16
Время обучения: 2070.764576435089 сек
Шаг № 17
Время обучения: 2200.1087102890015 сек
Шаг № 18
Время обучения: 2328.831901550293 сек
Шаг № 19
Время обучения: 2457.411291360855 сек
Шаг № 20
Время обучения: 2586.0409412384033 сек
Шаг № 21
Время обучения: 2714.84313416481 сек
Шаг № 22
В

- Time taken to train / Время работы модели doc2vec: <b> 7770.951511859894 s</b>
- Wall time: <b>2h 9min 33s</b>

# ===========================================================
### 2.3.1 Additionally, we can extract word2vec model from doc2vec model<br>Получение (извлечение) w2v модели из d2v

In [213]:
%%time

with open('RuNews100dw2v.txt', 'w', encoding='utf-8') as f:
    for word in tqdm_notebook(range(len(modeld2v.wv.index2word))):
        print(modeld2v.wv.index2word[word], ' '.join(map(str, modeld2v.wv.vectors[word])), file=f)  # Python 3.x
        sleep(0.01)

In [48]:
len(modeld2v.wv.index2word)

203399

In [181]:
# --- a sample word2vec feature ---
modeld2v.wv.vectors[100]

array([-1.31106102e+00, -1.74656892e+00, -3.16246413e-02,  3.07682961e-01,
        1.23358764e-01,  1.41161346e+00,  1.56246588e-01, -1.13048542e+00,
        2.63945274e-02,  6.38035893e-01,  2.05029917e+00, -9.53719318e-01,
        2.34890923e-01, -5.92913687e-01,  1.59463453e+00, -5.37049055e-01,
        7.18624711e-01, -1.11898690e-01, -1.04526830e+00,  5.49407363e-01,
        9.99868214e-01,  8.65986288e-01, -7.47275591e-01, -7.44117379e-01,
        1.53092587e+00, -5.52976310e-01,  3.30456674e-01,  5.91102004e-01,
       -9.95407343e-01, -1.60459816e+00, -8.78866136e-01,  3.03408980e-01,
       -6.73844099e-01,  6.03131354e-02,  5.93803763e-01, -1.94411606e-01,
       -1.11310911e+00, -5.22828847e-02,  8.60692143e-01,  9.80584681e-01,
        6.77737951e-01,  7.62811184e-01, -7.81033278e-01,  5.23834705e-01,
        9.60146010e-01, -3.46569180e-01, -1.70790577e+00,  1.44371748e+00,
       -4.13551092e-01, -4.54421639e-01, -2.91113108e-01, -3.76478165e-01,
       -1.04127300e+00,  

In [205]:
# --- a sample doc2vec feature / Просмотр первого вектора документов признаков ---
modeld2v.docvecs[1]

array([-1.8167664 , -1.5295626 ,  0.49334773,  0.16557695,  2.788194  ,
        0.5513849 ,  0.49569517,  2.502334  ,  2.0958338 ,  1.7315416 ,
        2.4732335 ,  4.3094463 ,  0.9542798 , -0.09378598,  1.293407  ,
        1.3053361 ,  0.5189122 , -1.8569682 , -2.1744719 , -0.5217152 ,
        2.594591  , -0.5637574 ,  0.1297457 , -0.26955035,  1.8756579 ,
       -2.1968243 , -2.246365  , -0.24344498,  3.1657422 , -0.99491507,
        2.3910112 ,  1.5235071 , -0.22778544,  1.4333862 ,  0.7881089 ,
        0.35705388,  2.907201  ,  0.20624931, -2.0854192 , -2.0690324 ,
       -0.47008103, -0.5385879 , -1.9149523 , -0.9514746 , -2.8571208 ,
        0.42290214,  2.260904  , -1.7204531 ,  1.4637091 ,  0.14459695,
       -2.8479514 , -2.1169395 , -1.9291022 , -0.5291996 ,  0.7535118 ,
       -1.1257833 , -1.5094233 ,  1.953258  , -1.1661766 ,  0.9460643 ,
        2.4747188 ,  1.9619635 , -1.2644751 , -1.8480142 , -1.8129939 ,
        1.2855972 ,  1.2380451 , -0.94859433,  1.5041687 , -1.38

In [209]:
train.index.values

array([ 15806,  45177, 121794, ...,  20463,  18638,  35683], dtype=int64)

# ===========================================================

## 2.4 Overview of similar document vectors / Просмотр схожих векторно документов

In [19]:
# --- Read documents / Прочитаем эти найденные документы ---
for i, similarity in enumerate(similar_docs):
    # --- To what extent is a particular document similar to 0 document / насколько этот документ похож на 0-ой документ ---
    print(similarity)  
    # --- document text / текст документа ---
    print(train['text_final'].iloc[i])  
    print('-' * 100)
    # --- document category / тема документа ---
    print('Категория: ', train['topic'].iloc[i])       
    print('-' * 100)

('108563', 0.5428193807601929)
найти обломок самолёт индонезийский авиакомпания пропасть экран радар январь год сообщить четверг индонезийский власть фрагмент хвостовой часть самолёт бортовый номер обнаружить местный рыбак вода недалеко остров сулавеси передавать обломок лежать расстояние километр населить пункт парепара метр береговой линия сообщить высокопоставленный представитель ввс индонезия руководящий поисковый операция напомнить борт самолёт выполнять рейс сурабайя остров ява манадо сулавеси находиться человек пассажир член экипаж надежда ктолибо пассажир самолёт выжить авиакатастрофа наиболее вероятный причина крушение предварительный дать считаться плохой погодный условие
----------------------------------------------------------------------------------------------------
Категория:  Мир
----------------------------------------------------------------------------------------------------
('62860', 0.5338842868804932)
уполномоченный президент россия право предприниматель борис т

## 2.4.1 Features and lables extraction / Извлечение векторов признаков и категорий

- <b>X_train</b> - векторы признаков <b>features_train</b>
- <b>y_train</b> - категории <b>labels_train</b>
- <b>X_test</b> - векторы признаков <b>features_test</b>
- <b>y_test</b> - категории <b>labels_test</b>

In [20]:
%%time

X_train = np.array([modeld2v.docvecs[str(i)] for i in range(len(tagged_tr))])
y_train = train.topic

X_test = np.array([modeld2v.infer_vector(tagged_test[i][0]) for i in range(len(tagged_test))])
y_test = test.topic

Wall time: 44.2 s


In [21]:
# --- quick check for dimensionality --- 
print(X_train.shape, X_test.shape)

(125624, 100) (22169, 100)


In [267]:
# --- document amount ---
len(modeld2v.docvecs.vectors_docs)

125624

In [68]:
from collections import Counter, defaultdict
from copy import deepcopy
import argparse

import sys

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import csr_matrix

### 2.4.2 Retrofitting

- <b>vectors</b> - trained vector document embeddings / обученные векторные представления (в нашем случае документов), 
- <b>neighbors</b> - neighbour objects это соседние точки (in original article) / (в оригинальной статье)
- <b>normalize</b> - normalizing function / функция нормализации по выбору
- <b>num_iters</b> - number of iterarions / количество итераций
- <b>alpha*</b> - alpha parameter for Retrofitting control / параметр по выбору, позволяющий контролировать степень ретрофиттинга
- <b>*</b> параметр альфа контролирует степень влияния процесса ретрофиттинга на изначально обученный эмбеддинг, позволяя изучить эффект от его действия

In [193]:
# --- Custom Retrofitting function (original author: Manaal Faruqui)
def retrofit(vectors, neighbors, normalize=False, num_iters=10, alpha=0.5):
    
    # --- N - number of records, D - dimensionality (100) ---
    N, D = vectors.shape

    # --- returns complete copy of vectors / возвращает полную копию vectors ---
    new_vectors = deepcopy(vectors) 
    
    # --- Normalization if needed / Нормализация, если есть необходимость ---
    if normalize:
        for c in range(N):
            vectors[c] /= np.sqrt((vectors[c]**2).sum() + 1e-6)
    # ------------------------------------------------------------------
    
    # retrofitting
    print("retrofitting", file=sys.stderr, flush=True)

    # --- Additional hyperparameter / Специальный гиперпараметр beta=alpha, here it is (beta = 1 - alpha) ---
    beta = 1-alpha

    for it in tqdm_notebook(range(num_iters)):
        
        # --- loop through each node / просмотр каждого узла (temporary dimensionality / временная размерность N = 5000)
        for c in range(N):
            instance_neighbours = neighbors[c]
            num_neighbours = len(instance_neighbours)
            
            # --- if no neighbours, pass - use data estimate ---
            if num_neighbours == 0:
                continue

            # --- the weight of the data estimate is the number of neighbours, plus the sum of all neighboring vectors, normalized ---
            new_vectors[c] = (alpha * (num_neighbours * vectors[c]) + beta * (new_vectors[instance_neighbours].sum(axis=0))) / (alpha * num_neighbours + beta * num_neighbours)
            
        sleep(0.01)
    return new_vectors

# --- additional function ---
def get_neighbors(y):
    value2id = {value: {idx for idx, v2 in enumerate(y) if v2 == value} for value in y}
    id2neighbors = {}
    for _, idx in value2id.items():
        for id_ in idx:
            id2neighbors[id_] = list(idx)
            id2neighbors[id_].remove(id_)
    return id2neighbors

In [ ]:
%%time
# --- additinal info ---
print(''' ======================================================================
        --- vectors        = X ---> modeld2v.docvecs.vectors_docs
        --- neighbours     = D ---> get_neighbors(y) ---> y = train["topic"].values
        --- normalize      = False (True)
        --- alpha          = 0.5
        ======================================================================
        --- N, D           = vectors.shape ---> [n x 100]
        ======================================================================
        --- После ретрофиттинга вес данных оценивается как 
        (количество соседей + сумма всех окружающих векторов, нормализованных)
        ======================================================================
        --- new_vectors - новые векторы признаков после ретрофиттинга
    ''')

# --- 50k raws retrofitting --- 
nrows = 50000

#for run in tqdm_notebook(range(10)):
#print('Итерация {}'.format(run+1), file=sys.stderr, flush=True)
    
# --- Feature vectors X for the first n records / Векторы признаков X на первые n записей [:nrows] ---
X_original_vectors = modeld2v.docvecs.vectors_docs[:nrows]
    
# --- Categories for the first n records / Категории на первые n записей --- 
y = train["topic"].values[:nrows]
    
# --- Get neighbors dictionary / Получить словарь соседей ---
D = get_neighbors(y_train)
    
# --- Apply Retrofitting / Ретрофиттинг эмбеддингов обучающей выборки
X_retrofitted_vectors = retrofit(X_original_vectors, D, normalize=False, num_iters=10, alpha=0.5)

        --- vectors        = X ---> modeld2v.docvecs.vectors_docs
        --- neighbours     = D ---> get_neighbors(y) ---> y = train["topic"].values
        --- normalize      = False (True)
        --- alpha          = 0.5
        --- N, D           = vectors.shape ---> [n x 100]
        --- После ретрофиттинга вес данных оценивается как 
        (количество соседей + сумма всех окружающих векторов, нормализованных)
        --- new_vectors - новые векторы признаков после ретрофиттинга
    


In [ ]:
# --- test code (currently deprecated) ---
'''size = 5000

for run in tqdm_notebook(range(10)):
    print('Итерация {}'.format(run+1), file=sys.stderr, flush=True)
    print('*' * 10, file=sys.stderr, flush=True)

    # выбираем количество статей на которых обучается выборка - X_train feature vectors
    subset_ids = np.random.choice(num_articles, size, replace=False)
    
    X = article_vectors[subset_ids]
    
    # labels = df[args.target].values
    y = labels[subset_ids]


    print('\tretrofitting', file=sys.stderr, flush=True)
    
    # Получить словарь соседей - get neighbor dictionary
    D = get_neighbors(y)
    
    
    # retrofit training embeddings
    X_retrofit = retrofit(X, D, normalize=False, num_iters=10, alpha=0.5)
    
    # train different models - Logistic Regression
    clf_embeddings = LogisticRegression()
    clf_retrofit_embeddings = LogisticRegression()


    print('\ttraining embedding model', file=sys.stderr, flush=True)
    clf_embeddings.fit(X, y)
    
    print('\ttraining retrofit embedding model', file=sys.stderr, flush=True)
    clf_retrofit_embeddings.fit(X_retrofit, y)
    

    # Test data
    mask = np.ones(num_articles, np.bool)
    mask[subset_ids] = 0
    Z = article_vectors[mask]
    gold = labels[mask]
    

    print('\tfinding translation matrices', file=sys.stderr, flush=True)
    translation_matrix_embeddings = np.linalg.lstsq(X, X_retrofit, rcond=None)[0]
    
    Z_retrofit = np.dot(Z, translation_matrix_embeddings)

    print('\tpredicting', file=sys.stderr, flush=True)
    
    # predictions
    predictions_embeddings = clf_embeddings.predict(Z)
    predictions_retrofit_embeddings = clf_retrofit_embeddings.predict(Z_retrofit)
    

    print('\tscoring', file=sys.stderr, flush=True)
    f1_embeddings = f1_score(gold, predictions_embeddings, average='micro')
    f1_retrofit_embeddings = f1_score(gold, predictions_retrofit_embeddings, average='micro')

    outcomes = [f1_embeddings, f1_retrofit_embeddings]
    print('\t{}'.format(outcomes))
    run_results.append(outcomes)
    
    sleep(0.01)
    '''

In [ ]:
# --- test code (currently deprecated) ---
'''run_results = [[5000] + np.array(run_results).mean(axis=0).tolist()]
results = pd.DataFrame(run_results, columns=['known', 'embeddings', 'retrofit'])
results.to_csv('{}{}_{}.csv'.format('retrofitted', 'topic', 5000))'''

## 3. Dimensionality reduction using t-SNE / снижение размерности

- Dimensinality reduction of original embeddings / Снижение размерности изначальных эмбеддингов
- Comparison original and retrofitted embeddings / Сравнение оригинальных векторных представлений:
<br><b>X_original_vectors & X_retrofitted_vectors</b>
- Parameter variation / Вариация параметра <b>perplexity</b>

In [258]:
%%time
nrows = 10000

tsne = TSNE(n_components=2, perplexity=40, metric='euclidean', verbose = 0, n_iter = 1000, init = "pca")

tsne_d2v = tsne.fit_transform(X_retrofitted_vectors[:nrows])

# --- add tSNE data to dataframe --- 
tsne_d2v_df = pd.DataFrame(data=tsne_d2v, columns=["x", "y"])  

tsne_d2v_df["topic"] = train["topic"].values[:nrows]
tsne_d2v_df["text_final"] = train["text_final"].values[:nrows]
tsne_d2v_df["topic_id"] = train["topic_id"].values[:nrows]
tsne_d2v_df["article_id"] = train.index.values[:nrows]

Wall time: 3min


In mathematical statistics, the Kullback–Leibler divergence (also called relative entropy) is a measure of how one probability distribution is different from a second, reference probability distribution.

In [211]:
# --- overview first rows of final df / посмотрим первые строки полученного датафрейма ---
tsne_d2v_df.head()

,x,y,topic,text_final,topic_id,article_id
0,-30.619671,-16.415527,Мир,найти обломок самолёт индонезийский авиакомпан...,1,15806
1,8.070693,8.936340,Россия,уполномоченный президент россия право предприн...,0,45177
2,-66.717995,-6.738235,Россия,независимый профсоюз горняк прекратить организ...,0,121794
3,-16.965836,4.323083,Россия,прокуратура москва категорически опровергнуть ...,0,108241
4,7.998756,32.268333,Спорт,махачкалинский анжи московский локомотив попас...,4,58275


In [54]:
tsne_d2v_df.text_parsed.str.encode('utf-8')

0       b' \xd0\xbd\xd0\xb5\xd1\x81\xd0\xba\xd0\xbe\xd...
1       b'\xd0\xb0\xd0\xbd\xd0\xb3\xd0\xbb\xd0\xb8\xd0...
2       b' \xd0\xb2\xd0\xbe\xd1\x81\xd0\xba\xd1\x80\xd...
3       b' \xd1\x85\xd0\xbe\xd0\xb4\xd0\xb5 \xd0\xbc\x...
4       b' \xd1\x84\xd0\xbe\xd0\xbd\xd0\xb5 \xd0\xba\x...
                              ...                        
4995    b'\xd1\x82\xd0\xb2\xd0\xb5\xd1\x80\xd1\x81\xd0...
4996    b'\xd0\xb8\xd0\xb7\xd0\xb1\xd1\x80\xd0\xb0\xd0...
4997    b' \xd1\x8f\xd0\xbd\xd0\xb2\xd0\xb0\xd1\x80\xd...
4998    b' \xd1\x82\xd1\x83\xd0\xb0\xd0\xbb\xd0\xb5\xd...
4999    b'\xd1\x81\xd0\xbe\xd1\x82\xd1\x80\xd1\x83\xd0...
Name: text_parsed, Length: 5000, dtype: object

In [212]:
# --- df size / размерность полученного датафрейма ---
tsne_d2v_df.shape

(10000, 6)

In [25]:
# --- visualization with bokeh ---
from bokeh.models import ColumnDataSource, LabelSet, HoverTool
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook, reset_output
from bokeh.palettes import d3
import bokeh.models as bmo

In [218]:
len(tsne_d2v_df['topic_id'].unique())

18

In [259]:
# --- all visualization can be found in a separate folder with images ---

plot_d2v = bp.figure(plot_width = 850, plot_height = 600, 
                       title = "Ретрофиттинг - Снижение размерноси t-SNE doc2vec - perplexity = 120 - Первые 10,000 статей",
                       tools = "pan, wheel_zoom, box_zoom, reset, hover, previewsave",
                       x_axis_type = "linear", y_axis_type = "linear", min_border = 1)

colormap = np.array(["#206303",  # 1 темно-зеленый
                     "#0000cc",  # 2 темно-синий
                     "#630363",  # 3 темно-фиолетовый
                     "#52fa52",  # 4 светло-зеленый
                     "#527cfa",  # 5 светло-синий
                     "#f953f9",  # 6 светло-фиолетовый
                     "#f95353",  # 7 красный
                     "#ffff1a",  # 8 желтый
                     "#f9a653",  # 9 оранжевый
                     "#944d05",  # 10 коричневый
                     "#08dda8",  # 11 бирюзовый
                     "#a1aaa8",  # 12 серый
                     "#ff1a8c",  # 13 темно-розовый
                     "#000000",  # 14 черный
                     "#999900",  # 15 темно-желтый
                     "#cc00cc",  # 16 лиловый
                     "#2d8659",  # 17 лес
                     "#00bfff",  # 18 небесный
                     "#5AF109",
                     "#056794",
                     "#FFF176"])

source = ColumnDataSource(data = dict(x = tsne_d2v_df["x"], 
                                      y = tsne_d2v_df["y"],
                                      color = colormap[tsne_d2v_df["topic_id"]],
                                      text = tsne_d2v_df["text_final"],
                                      topic_id = tsne_d2v_df["topic_id"],
                                      topic = tsne_d2v_df["topic"],
                                      article_id = tsne_d2v_df["article_id"]))

plot_d2v.scatter(x = "x", 
                   y = "y", 
                   color="color",
                   size = 3,
                   legend = "topic",
                   source = source,
                   alpha = 0.8)


hover = plot_d2v.select(dict(type = HoverTool))

hover.tooltips = {"id категории":  "@topic_id", 
                  "Координаты":    "@x & @y", 
                  "категория":     "@topic",
                  "id статьи":     "@article_id"}

show(plot_d2v)

## 4. Saving results with pickle / Сохранение результатов

In [ ]:
# features_train original embeddings
with open('[02] Data/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('[02] Data/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test original embeddings
with open('[02] Data/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('[02] Data/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# features_train retrofitted embeddings
with open('[02] Data/features_train_retrofitted.pickle', 'wb') as output:
    pickle.dump(X_retrofitted_vectors, output)
    
# features_test retrofitted embeddings
with open('[02] Data/features_test_retrofitted.pickle', 'wb') as output:
    pickle.dump(X_retrofitted_vectors_test, output)